In [3]:
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
from dotenv import dotenv_values
from datasets import load_dataset, Dataset
from utils.data_preprocessor import DataPreprocessor
from utils.postprocessor import DataPostprocessor
from utils.evaluator import Evaluator
from config import config

HF_TOKEN = dotenv_values(".env.base")['HF_TOKEN']
# If the dataset is gated/private, make sure you have run huggingface-cli login

load_model = True
if load_model:
    checkpoint_adapter = 'ferrazzipietro/Mistral-7B-Instruct-v0.2_adapters_it.layer1_v0.2_wandblog'
    bnb_config = BitsAndBytesConfig(
        
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
    )

    base_model_reload = AutoModelForCausalLM.from_pretrained(
        config.BASE_MODEL_CHECKPOINT, low_cpu_mem_usage=True,
        quantization_config = bnb_config,
        return_dict=True,  load_in_4bit=True, #torch_dtype=torch.float16,
        device_map= "auto")

    adp = 'ferrazzipietro/Mistral-7B-Instruct-v0.2_adapters_it.layer1_v0.2_wandblog'
    merged_model = PeftModel.from_pretrained(base_model_reload, adp, token=HF_TOKEN, device_map="auto")
    
tokenizer = AutoTokenizer.from_pretrained(config.BASE_MODEL_CHECKPOINT, add_eos_token=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

from datasets import load_dataset
dataset = load_dataset("ferrazzipietro/e3c-sentences", token=HF_TOKEN)
dataset = dataset[config.TRAIN_LAYER]
preprocessor = DataPreprocessor()
dataset = preprocessor.preprocess_data_one_layer(dataset)
train_data, val_data, _ = preprocessor.split_layer_into_train_val_test_(dataset, config.TRAIN_LAYER)

adapter_config.json: 100%|██████████| 552/552 [00:00<00:00, 431kB/s]
adapter_model.safetensors: 100%|██████████| 92.3M/92.3M [00:03<00:00, 28.4MB/s]
Map: 100%|██████████| 135/135 [00:00<00:00, 6177.11 examples/s]


In [ ]:
class HyperparametersSearcher():

    def __init__(self, preprocessor:DataPreprocessor, data: Dataset) -> None:
        self.preprocessor = preprocessor
        self.csvs_dict = {}

        pass

def hyperparameters_search_helper(max_new_tokens: bool, similar_is_equal: bool, similar_is_equal_threshold: int):
    if max_new_tokens:
        test_data = load_dataset('csv', data_files='data/postprocessor_output_small_max_new_tokens.csv', split='train')
    evaluator = Evaluator(preprocessor, test_data)
    results = evaluator.generate_evaluation_table(similar_is_equal=True, similar_is_equal_threshold=80)
    results['precision'], results['recall'], results['f1']

In [ ]:
postprocessor = DataPostprocessor(val_data, preprocessor, 2, 'en', tokenizer)
postprocessor.test_data = load_dataset('csv', data_files='data/postprocessor_output_small_max_new_tokens.csv', split='train')
evaluator = Evaluator(preprocessor, postprocessor.test_data)
results = evaluator.generate_evaluation_table(similar_is_equal=True, similar_is_equal_threshold=80)
results['precision'], results['recall'], results['f1']

In [ ]:
postprocessor = DataPostprocessor(val_data, preprocessor, 2, 'eng', tokenizer)


In [118]:
postprocessor = DataPostprocessor(val_data, preprocessor, 2, 'eng', tokenizer)
postprocessor.test_data = load_dataset('csv', data_files='data/postprocessor_output_small_max_new_tokens.csv', split='train')
evaluator = Evaluator(preprocessor, postprocessor.test_data)
results = evaluator.generate_evaluation_table(similar_is_equal=True, similar_is_equal_threshold=80)
results['precision'], results['recall'], results['f1']

ORIGINAL model_response:  []
GROUND THRUTH:  ['hypertension', 'dyslipidemia', 'diagnosed', 'mellitus', 'referred', 'hypokalemia', 'new-onset diabetes mellitus', 'A 46-year-old man', '4-months', '1-month']
NEW model_response:  [] 


ORIGINAL model_response:  ['study', 'tests', 'indicated']
GROUND THRUTH:  ['study', 'tests', 'indicated', 'ECS']
NEW model_response:  ['study', 'tests', 'indicated'] 


ORIGINAL model_response:  []
GROUND THRUTH:  ['findings', 'pointed', 'primary right parotid malignancy', 'liver metastases']
NEW model_response:  [] 


ORIGINAL model_response:  ['parotidectomy', 'examination', 'confirmed', 'ACC', 'The patient']
GROUND THRUTH:  ['parotidectomy', 'examination', 'confirmed', 'ACC', 'The patient']
NEW model_response:  ['parotidectomy', 'examination', 'confirmed', 'ACC', 'The patient'] 


ORIGINAL model_response:  ['manage']
VERO 2
GROUND THRUTH:  ['hypercortisolism', 'managed']
NEW model_response:  ['manage'] 


ORIGINAL model_response:  []
GROUND THRUTH:  ['hyp

(0.6906923950056754, 0.3299891540130152, 0.4466055045871559)

In [114]:
postprocessor = DataPostprocessor(val_data, preprocessor, 2, 'eng', tokenizer)
postprocessor.test_data = load_dataset('csv', data_files='data/postprocessor_output_small_max_new_tokens.csv', split='train')
evaluator = Evaluator(preprocessor, postprocessor.test_data)
results = evaluator.generate_evaluation_table(similar_is_equal=True, similar_is_equal_threshold=80)
results['precision'], results['recall'], results['f1']

ORIGINAL model_response:  []
GROUND THRUTH:  ['hypertension', 'dyslipidemia', 'diagnosed', 'mellitus', 'referred', 'hypokalemia', 'new-onset diabetes mellitus', 'A 46-year-old man', '4-months', '1-month']
NEW model_response:  [] 


ORIGINAL model_response:  ['study', 'tests', 'indicated']
GROUND THRUTH:  ['study', 'tests', 'indicated', 'ECS']
NEW model_response:  ['study', 'tests', 'indicated'] 


ORIGINAL model_response:  []
GROUND THRUTH:  ['findings', 'pointed', 'primary right parotid malignancy', 'liver metastases']
NEW model_response:  [] 


ORIGINAL model_response:  ['parotidectomy', 'examination', 'confirmed', 'ACC', 'The patient']
GROUND THRUTH:  ['parotidectomy', 'examination', 'confirmed', 'ACC', 'The patient']
NEW model_response:  ['parotidectomy', 'examination', 'confirmed', 'ACC', 'The patient'] 


ORIGINAL model_response:  ['manage']
VERO 2
GROUND THRUTH:  ['hypercortisolism', 'managed']
NEW model_response:  ['managed'] 


ORIGINAL model_response:  []
GROUND THRUTH:  ['hy

(0.7093484419263456, 0.33947939262472887, 0.4591967724188521)

In [103]:
postprocessor = DataPostprocessor(val_data, preprocessor, 2, 'eng', tokenizer)
postprocessor.test_data = load_dataset('csv', data_files='data/postprocessor_output_small_max_new_tokens.csv', split='train')
cleaned_outputs = []
for output in postprocessor.test_data['model_responses']:
    cleaned_outputs.append(postprocessor._postprocess_model_output(output))
postprocessor.test_data = postprocessor.test_data.remove_columns('model_responses')
postprocessor.test_data = postprocessor.test_data.add_column('model_responses', cleaned_outputs)
evaluator = Evaluator(preprocessor, postprocessor.test_data)
results = evaluator.generate_evaluation_table(similar_is_equal=True, similar_is_equal_threshold=80)
results['precision'], results['recall'], results['f1']

[{'entity': 'study'}, {'entity': 'tests'}, {'entity': 'indicated'}]
[{'entity': 'parotidectomy'}, {'entity': 'examination'}, {'entity': 'confirmed'}, {'entity': 'ACC'}, {'entity': 'ACC'}, {'entity': 'The patient'}]
[{'entity': 'manage'}]
[{'entity': 'history'}, {'entity': 'surgery'}, {'entity': 'MNG'}, {'entity': 'her mother, sisters and cousins'}]
[{'entity': 'signs'}, {'entity': 'compression'}, {'entity': 'the patient'}]
[{'entity': 'took'}, {'entity': 'The mass'}, {'entity': 'the front and the two sides of the neck'}]
[{'entity': 'covered'}]
[{'entity': 'limited'}, {'entity': 'circulation'}, {'entity': 'the neck'}]
[{'entity': 'goiter'}, {'entity': 'measured'}, {'entity': '18 x 11 cm'}]
[{'entity': 'The mass'}]
[{'entity': 'tests'}, {'entity': 'T 3'}, {'entity': 'T 4'}, {'entity': 'TSH'}, {'entity': 'normal'}]
[{'entity': 'radiography'}, {'entity': 'showed'}, {'entity': 'opacifiction'}, {'entity': 'strewn with microcalcifications'}, {'entity': 'associated with a right eccentricity'}

(0.68993993993994, 0.49837310195227763, 0.5787153652392947)

In [112]:
postprocessor = DataPostprocessor(val_data, preprocessor, 2, 'eng', tokenizer)
postprocessor.test_data = load_dataset('csv', data_files='data/postprocessor_output.csv', split='train')
cleaned_outputs = []
for output in postprocessor.test_data['model_responses']:
    cleaned_outputs.append(postprocessor._postprocess_model_output(output))
postprocessor.test_data = postprocessor.test_data.rename_column('model_responses', 'model_responses_original')
postprocessor.test_data = postprocessor.test_data.add_column('model_responses', cleaned_outputs)
evaluator = Evaluator(preprocessor, postprocessor.test_data)
results = evaluator.generate_evaluation_table(similar_is_equal=True, similar_is_equal_threshold=80)
results['precision'], results['recall'], results['f1']

[{'entity': 'diagnosed'}, {'entity': 'unveiled'}, {'entity': 'referred'}, {'entity': 'hypokalemia'}, {'entity': 'hypotension'}, {'entity': 'dyslipidemia'}, {'entity': 'diabetes mellitus'}, {'entity': 'hypertension'}, {'entity': 'dyslipidemia'}, {'entity': 'diabetes mellitus'}, {'entity': 'A 46-year-old man'}, {'entity': '4-months'}, {'entity': '1-month'}]
[{'entity': 'study'}, {'entity': 'tests'}, {'entity': 'indicated ECS'}]
[{'entity': 'findings'}, {'entity': 'pointed'}, {'entity': 'malignancy'}, {'entity': 'parotid malignancy'}, {'entity': 'liver metastases'}, {'entity': 'primary right parotid malignancy'}, {'entity': 'liver metastases'}]
[{'entity': 'parotidectomy'}, {'entity': 'examination'}, {'entity': 'confirmed'}, {'entity': 'ACC'}, {'entity': 'ACC'}, {'entity': 'The patient'}]
[{'entity': 'managed'}, {'entity': 'hypercortisolism'}, {'entity': 'hypercortisolism'}, {'entity': 'metyrapone'}, {'entity': 'ketoconazole'}, {'entity': 'lanreotide'}]
[{'entity': 'hypertensive'}, {'enti

(0.574108818011257, 0.49783080260303686, 0.533255881498693)

In [113]:
postprocessor = DataPostprocessor(val_data, preprocessor, 2, 'eng', tokenizer)
postprocessor.test_data = load_dataset('csv', data_files='data/postprocessor_output.csv', split='train')
cleaned_outputs = []
for output in postprocessor.test_data['model_responses']:
    cleaned_outputs.append(postprocessor._postprocess_model_output(output))
postprocessor.test_data = postprocessor.test_data.rename_column('model_responses', 'model_responses_original')
postprocessor.test_data = postprocessor.test_data.add_column('model_responses', cleaned_outputs)
evaluator = Evaluator(preprocessor, postprocessor.test_data)
results = evaluator.generate_evaluation_table(similar_is_equal=False, similar_is_equal_threshold=80)
results['precision'], results['recall'], results['f1']

[{'entity': 'diagnosed'}, {'entity': 'unveiled'}, {'entity': 'referred'}, {'entity': 'hypokalemia'}, {'entity': 'hypotension'}, {'entity': 'dyslipidemia'}, {'entity': 'diabetes mellitus'}, {'entity': 'hypertension'}, {'entity': 'dyslipidemia'}, {'entity': 'diabetes mellitus'}, {'entity': 'A 46-year-old man'}, {'entity': '4-months'}, {'entity': '1-month'}]
[{'entity': 'study'}, {'entity': 'tests'}, {'entity': 'indicated ECS'}]
[{'entity': 'findings'}, {'entity': 'pointed'}, {'entity': 'malignancy'}, {'entity': 'parotid malignancy'}, {'entity': 'liver metastases'}, {'entity': 'primary right parotid malignancy'}, {'entity': 'liver metastases'}]
[{'entity': 'parotidectomy'}, {'entity': 'examination'}, {'entity': 'confirmed'}, {'entity': 'ACC'}, {'entity': 'ACC'}, {'entity': 'The patient'}]
[{'entity': 'managed'}, {'entity': 'hypercortisolism'}, {'entity': 'hypercortisolism'}, {'entity': 'metyrapone'}, {'entity': 'ketoconazole'}, {'entity': 'lanreotide'}]
[{'entity': 'hypertensive'}, {'enti

(0.56375, 0.4891540130151844, 0.5238095238095238)